# Mistral-7B
- To DO???
- Performance
- 7 Billion parameters, 24GB at least GPU (VRAM)
- Should be best of all selected models

In [ ]:
from datasets import load_dataset
import os 
import pandas as pd
import sys
    
NOTEBOOK_DIR = os.path.dirname(os.path.abspath("__file__"))
print(NOTEBOOK_DIR)

In [ ]:
# https://huggingface.co/datasets/gretelai/synthetic_text_to_sql
dataset = load_dataset("gretelai/synthetic_text_to_sql")

print("Available data subsets:", dataset.keys())
print("Features: ")
for k, v in dataset["train"][0].items():
    print(f"---> {k:30}: {v}")

In [ ]:
from typing import List

def create_sharegpt_format(df: pd.DataFrame) -> List[dict]:
    result = []
    for _, row in df.iterrows():
        result.append({
            "conversations": [
                {"from": "user", "value": row["text"]},
                {"from": "assistant", "value": row["sql"]}
            ]
        })
    return result

os.makedirs("data", exist_ok=True)

for split_name in ["train", "test"]:
    df = pd.DataFrame(dataset[split_name])
    sharegpt_data = create_sharegpt_format(df)

    output_file = Path(f"data/dataset_{split_name}.json")
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(sharegpt_data, f, indent=2, ensure_ascii=False)

    print(f" Saved: {output_file} — {len(sharegpt_data)} records")


In [ ]:

!mkdir -p $NOTEBOOK_DIR/LLaMA-Factory/data/
!cp -r $NOTEBOOK_DIR/data/* $NOTEBOOK_DIR/LLaMA-Factory/data/

In [ ]:
training_args = {
    "model_name_or_path": "TinyLlama/TinyLlama-1.1B-Chat",
    "template": "chatml",
    "finetuning_type": "lora",
    "dataset": "dataset_train",
    "dataset_dir": "./data",
    "val_set_size": 0.01,
    "output_dir": "output/tinyllama-sql",
    "cutoff_len": 1024,
    "fp16": True,
    "lora_rank": 8,
    "lora_alpha": 16,
    "gradient_checkpointing": True,
    "gradient_accumulation_steps": 2,
    "per_device_train_batch_size": 2,
    "num_train_epochs": 3,
    "learning_rate": 2e-5,
    "lr_scheduler_type": "cosine",
    "save_steps": 250,
    "logging_steps": 10,
    "report_to": "none",
    "overwrite_output_dir": True,
    "plot_loss": True,
    "warmup_steps": 50,
    "flash_attn": False,
    "lora_target": "q_proj,v_proj",
    "use_dora": True,
    "stage": "sft"
}

config_path = f"{NOTEBOOK_DIR}/LLaMA-Factory/train_tinyllama.json"
with open(config_path, "w", encoding="utf-8") as f:
    json.dump(training_args, f, indent=2)
print(f"Config saved to: {config_path}")

In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git || echo "Already cloned"
%cd $NOTEBOOK_DIR/LLaMA-Factory
%ls

In [ ]:
# %cd $NOTEBOOK_DIR/LLaMA-Factory
# !CUDA_VISIBLE_DEVICES=0 python src/train_bash.py --config_file train_tinyllama.json